In [1]:
import glob
import tensorflow as tf
# # from tensorflow.python.ops import control_flow_ops
# from tqdm import tqdm
from progressbar import ProgressBar
import h5py
import matplotlib.pyplot as plt


I=0
def _parse_function(example_proto):
    print('1')
    features = {"X": tf.FixedLenFeature((3*257), tf.float32),
              "Y": tf.FixedLenFeature((257), tf.float32)}
    parsed_features = tf.parse_single_example(example_proto, features)
    print("i was here")
    print('2')
    return parsed_features["X"], parsed_features["Y"]

def rbm_layer(n_visible, n_hidden, num_epochs, num_cases, lr, lrh, ws, bs, layer_n, len_data, directories):
    Data_path = directories[0]
    tfrecord_folder_parent = directories[1]
    tfrecord_folder = directories[2]
    
    tfrecord_path_x = os.path.normpath(os.path.join(Data_path,tfrecord_folder_parent,tfrecord_folder))
    sorted_names_x = natsorted(os.listdir(tfrecord_path_x))
    trainfilenames_x = []
    for i in sorted_names_x:
        trainfilenames_x.append(os.path.normpath(os.path.join(tfrecord_path_x,i)))
#     filenames_x = tf.placeholder(tf.string, shape=[None])
#     dataset_x = tf.data.TFRecordDataset(filenames_x)
    dataset_x = tf.data.TFRecordDataset(trainfilenames_x)
    dataset_x = dataset_x.map(_parse_function)  # Parse the record into tensors.
#     dataset_x = dataset_x.repeat()  # Repeat the input indefinitely.
    dataset_x = dataset_x.batch(num_cases)
#     iterator_x = dataset_x.make_initializable_iterator()
    
    weightcost  = 0.0002
    initialmomentum  = 0.5
    finalmomentum    = 0.9
    numcases = 128
    W_adder  = tf.zeros((n_visible,n_hidden),dtype=tf.dtypes.float32)
    bh_adder = tf.zeros((1,n_hidden),dtype=tf.dtypes.float32)
    bv_adder = tf.zeros((1,n_visible),dtype=tf.dtypes.float32)
    
    W  = tf.Variable(tf.random_normal([n_visible, n_hidden], mean=0, stddev=0.01), name="W") #The weight matrix that stores the edge weights
    bh = tf.Variable(tf.zeros([1, n_hidden],  tf.float32, name="bh")) #The bias vector for the hidden layer
    bv = tf.Variable(tf.zeros([1, n_visible],  tf.float32, name="bv")) #The bias vector for the visible layer

    def sample(probs):
        #Takes in a vector of probabilities, and returns a random vector of 0s and 1s sampled from the input vector
#         return tf.floor(probs + tf.random_uniform(tf.shape(probs), 0, 1))
        return tf.cast(probs > tf.random_uniform(tf.shape(probs), 0, 1),tf.float32)
        
    count=0
    pbar = ProgressBar()
    for epoch in pbar(range(num_epochs)):
        count+=1
        incount=0
        errsum = 0
        for x in dataset_x:

                    
            poshidprobs = tf.sigmoid(tf.matmul(x[0], W) + bh) #Propagate the visible values to sample the hidden values
            poshidstates = sample(poshidprobs)
            #128*512
            #Next, we update the values of W, bh, and bv, based on the difference between the samples that we drew and the original values
            posprods  = tf.matmul(tf.transpose(x[0]),poshidprobs) #771*512
            poshidacts = tf.reduce_sum(poshidprobs, axis=0) #(512)
#             print(poshidprobs)
#             print(poshidacts.numpy())
            posvisacts = tf.reduce_sum(x[0],axis=0)#771
            
            negdata   = tf.sigmoid(tf.matmul(poshidstates, tf.transpose(W)) + bv)#128*711
            neghidprobs   = tf.sigmoid(tf.matmul(negdata, W) + bh) #128*512
            negprods  = tf.matmul(tf.transpose(negdata),neghidprobs); #771*512##################################
            neghidacts = tf.reduce_sum(neghidprobs, axis=0)
            negvisacts = tf.reduce_sum(negdata, axis=0)
            
            if incount%100==0:
                print(incount)
            incount+=1
            if epoch>5:
                m=finalmomentum
            else:
                m=initialmomentum
#             [posprods,negprods,posvisacts,negvisacts,poshidacts,neghidacts] = gibbs_step(x[0])
            W_adder = (m * W_adder)+ (lr*(posprods-negprods)/numcases)-(weightcost*W)
            bv_adder = (m * bv_adder)+ ((lrh/numcases)*(posvisacts-negvisacts))
            bh_adder = (m * bh_adder)+ ((lrh/numcases)*(poshidacts-neghidacts))
#             print(posprods)
#             print(poshidacts-neghidacts)
#             print(bv_adder)
            #When we do sess.run(updt), TensorFlow will run all 3 update steps
            updt = [W.assign_add(W_adder).numpy(), bv.assign_add(bv_adder).numpy(), bh.assign_add(bh_adder).numpy()] 
            err= tf.reduce_sum(tf.reduce_sum((x[0]-negdata)**2 ,axis =0),axis=0)
            errsum = err + errsum;
        print(errsum)
#         print(count)
    return updt




In [15]:
# import libraries.
import numpy as np
from sklearn.utils import shuffle
import tensorflow as tf
import datetime
tf.enable_eager_execution()
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
# import keras
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from pystoi.stoi import stoi
import h5py
import sys
######################
#import libraries.
import matplotlib.pyplot as plt
from tabulate import tabulate
import time
import os
import librosa
from librosa.core import stft, istft
####import sounddevice as sd
import time
print('imported')
# #######################
Data_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff'
# Data_path = os.getcwd()
tfrecord_folder_parent = 'tfrecord_files'
tfrecord_folder = 'tfrecord_files_norm_small'
tfrecord_val_folder = 'validation_norm_small'
ckpt_folder = '5'
dirs = [Data_path, tfrecord_folder_parent, tfrecord_folder]
 
# len_data = (684108, 257)
# len_data = (2197278, 257)
# val_len = (97278,257)
len_data = (200000, 257)
val_len = (100000,257)
w=3
h = [512]
global batch_size
batch_size = 128
#######################
#define reconstruct function to reconstruct sound from framed signal.
def reconstruct(wave,angle):
    recon = np.sqrt(np.power(10, wave))
    recon1 = recon*np.cos(angle)+recon*np.sin(angle)*1j
    recon = librosa.core.istft((recon1.T), hop_length=256, win_length=512, window='hann')
    return recon

########################
visible = w*len_data[1]
hidden = h[0]
# visible1 = h[0]
# hidden1 = h[1]
# visible2 = h[1]
# hidden2 = len_data[1]

# layer1 = rbm_layer(visible, hidden, 10, batch_size, 0.1, 0.1, [np.eye(visible,hidden)], [np.zeros((1,visible))], 1, len_data[0],dirs)
# layer2 = rbm_layer(visible1, hidden1, 50, batch_size, 0.01, [np.eye(visible,hidden),layer1[0]], [np.zeros((1,visible)),layer1[2]], 2, len_data[0], dirs)
# layer3 = rbm_layer(visible2, hidden2, 50, batch_size, 0.01, [np.eye(visible),layer1[0],layer2[0]], [np.zeros((1,visible)),layer1[2],layer2[2]], 3, len_data[0], dirs)

###############################
#######################
I=0

# epochs_num=50
global datalen
datalen=len_data[0]


seed = 7
rate1 = 0.1
rate2 = 0.2
buffersize = 1000
from tensorflow.keras.layers import Activation
# from keras.layers import Activation
np.random.seed(seed)
act1 = layers.LeakyReLU(alpha=0.1)
model = Sequential()
# model.add(layers.Dropout(rate1, noise_shape=None, seed=None))
# ,kernel_regularizer=regularizers.l2(0.001)
# model.add(Dense(h[0], input_dim = w*len_data[1]))
model.add(Dense(h[0], input_dim = w*len_data[1], kernel_initializer= tf.constant_initializer(layer1[0]), bias_initializer = tf.constant_initializer(layer1[2])))
# , kernel_initializer= tf.constant_initializer(layer1[0]), bias_initializer = tf.constant_initializer(layer1[2])
# tf.constant_initializer(layer1[0])
# tf.constant_initializer(layer1[2])
model.add(BatchNormalization())
# model.add(act1)
model.add(Activation('sigmoid'))
# act2=layers.LeakyReLU(alpha=0.1)
# model.add(layers.Dropout(rate2, noise_shape=None, seed=None))
# model.add(Dense(h[1]))
# model.add(act2)
# act3=layers.LeakyReLU(alpha=0.1)
# # model.add(layers.Dropout(rate, noise_shape=None, seed=None))
# model.add(Dense(h[2]))
# model.add(act3)
# act=layers.LeakyReLU(alpha=0.01)
model.add(Dense(len_data[1]))
#############################################
import os
from natsort import natsorted

def _parse_function(example_proto):
    features = {"X": tf.FixedLenFeature((3*257), tf.float32),
              "Y": tf.FixedLenFeature((257), tf.float32)}
    parsed_features = tf.parse_single_example(example_proto, features)
    return parsed_features["X"], parsed_features["Y"]

tfrecord_path = os.path.normpath(os.path.join(Data_path,tfrecord_folder_parent,tfrecord_folder))
sorted_names = natsorted(os.listdir(tfrecord_path))
trainfilenames = []
for i in sorted_names:
    trainfilenames.append(os.path.normpath(os.path.join(tfrecord_path,i)))
# filenames = tf.placeholder(tf.string, shape=[None])
# dataset = tf.data.TFRecordDataset(filenames)
dataset = tf.data.TFRecordDataset(trainfilenames)
dataset = dataset.map(_parse_function)  # Parse the record into tensors.
dataset = dataset.repeat()  # Repeat the input indefinitely.
dataset = dataset.batch(batch_size)
dataset = dataset.shuffle(buffersize)
# iterator = dataset.make_initializable_iterator()

tfrecord_path_val = os.path.normpath(os.path.join(Data_path,tfrecord_folder_parent,tfrecord_val_folder))
sorted_names_val = natsorted(os.listdir(tfrecord_path_val))
trainfilenames_val = []
for i in sorted_names_val:
    trainfilenames_val.append(os.path.normpath(os.path.join(tfrecord_path_val,i)))
# filenames_val = tf.placeholder(tf.string, shape=[None])
# dataset_val = tf.data.TFRecordDataset(filenames_val)
dataset_val = tf.data.TFRecordDataset(trainfilenames_val)
dataset_val = dataset_val.map(_parse_function)  # Parse the record into tensors.
dataset_val = dataset_val.repeat()  # Repeat the input indefinitely.
dataset_val = dataset_val.batch(128)
# iterator_val = dataset_val.make_initializable_iterator()

epochs_num = 50
steps = len_data[0] // batch_size
val_steps = val_len[0] // batch_size
# You can feed the initializer with the appropriate filenames for the current
# phase of execution, e.g. training vs. validation.
# next_elem = iterator_val.get_next()
# Initialize `iterator` with training data.

if not os.path.exists(os.path.join(Data_path,"checkpoints",ckpt_folder)):
    os.makedirs(os.path.join(Data_path,"checkpoints",ckpt_folder))

print(datetime.datetime.now())
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     sess.run(iterator.initializer, feed_dict={filenames: trainfilenames})
#     sess.run(iterator_val.initializer, feed_dict={filenames_val: trainfilenames_val})
print("initialized")
checkpoint_path = os.path.normpath(os.path.join(Data_path,"checkpoints",ckpt_folder,"weights.{epoch:02d}.hdf5"))
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_best_only=True, save_weights_only=True)
        # Save weights, every 5-epochs.
#         period=1)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30)
# opt = tf.keras.optimizers.Adamax()
# opt = tf.train.AdamOptimizer()
opt = tf.keras.optimizers.SGD()
model.compile(loss='mean_squared_error', optimizer=opt)
history = model.fit( dataset, steps_per_epoch=steps,epochs=epochs_num, callbacks = [cp_callback,early_stop], verbose=1,validation_data=dataset_val,validation_steps=val_steps)
#     model.save(os.path.normpath(os.path.join(Data_path, 'models', "model_3h_dataset.h5")))
#     tf.keras.models.save_model(model, os.path.normpath(os.path.join(Data_path, 'models', "model_3h_dataset.h5")))
#     model.save_weights(os.path.normpath(os.path.join(Data_path, 'models', "model_3h_dataset.h5")))
model_json = model.to_json()
with open(os.path.normpath(os.path.join(Data_path, 'models', "model_"+ckpt_folder+".json")), "w") as json_file:
    json_file.write(model_json)
# # serialize weights to HDF5
model.save_weights(os.path.normpath(os.path.join(Data_path, 'models', "model_"+ckpt_folder+".h5")))
print("Saved model to disk")
    
print(datetime.datetime.now())
%matplotlib inline
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()
plt.savefig(os.path.normpath(os.path.join(Data_path,'images',ckpt_folder+'.png')))
# model_json = model.to_json()
# with open("model_10h_dataset.json", "w") as json_file:
#     json_file.write(model_json)
# model.save_weights("model_10h_dataset.h5")
# print("Saved model to disk")


imported
2019-07-11 13:21:18.328357
initialized
Epoch 1/50
781/781 [==============================] - 8s 10ms/step - loss: 2.3806

Epoch 00001: val_loss improved from inf to 2.38055, saving model to D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\checkpoints\5\weights.01.hdf5
1562/1562 [==============================] - 40s 26ms/step - loss: 3.2202 - val_loss: 2.3806
Epoch 2/50
781/781 [==============================] - 8s 10ms/step - loss: 2.3697

Epoch 00002: val_loss improved from 2.38055 to 2.36968, saving model to D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\checkpoints\5\weights.02.hdf5
1562/1562 [==============================] - 39s 25ms/step - loss: 2.8382 - val_loss: 2.3697
Epoch 3/50
781/781 [==============================] - 9s 11ms/step - loss: 2.3622

Epoch 00003: val_loss improved from 2.36968 to 2.36217, saving model to D:\studies\university\thesis\speech_separation_codes\du16\donesomestuff\checkpoints\5\weights.03.hdf5

KeyboardInterrupt: 

In [17]:
np.savetxt(os.path.normpath(os.path.join(Data_path,'results','rbm','test','layers10.txt')),layer1[0])
np.savetxt(os.path.normpath(os.path.join(Data_path,'results','rbm','test','layers11.txt')),layer1[1])
np.savetxt(os.path.normpath(os.path.join(Data_path,'results','rbm','test','layers12.txt')),layer1[2])

In [4]:
write_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff/10hdata'
mixed_folder = os.path.normpath(os.path.join(write_path,'ftr_refrmd_10h_norm2'))
h5f = h5py.File(mixed_folder+'.hdf5','r')
ftr = h5f['ftr_refrmd_10h_norm2'][0:126]
h5f.close()

In [5]:
hidden1 = np.matmul(tf.abs(ftr)/5,layer1[0].numpy())+layer1[2].numpy()
# hidden2 = np.matmul(hidden1,layer2[0])+layer2[2]
# hidden3 = np.matmul(hidden2,layer3[0])+layer3[2]
hidden1_b = np.matmul(hidden1,layer1[0].numpy().T)+layer1[1].numpy()
# hidden2_b = np.matmul(hidden3_b,layer2[0].T)+layer2[1]
# hidden1_b = np.matmul(hidden2_b,layer1[0].T)+layer1[1]

In [9]:
layer1[0]

<tf.Variable 'UnreadVariable' shape=(771, 512) dtype=float32, numpy=
array([[-0.01466631, -0.00084128,  0.02330974, ...,  0.01120422,
        -0.02315165,  0.00434515],
       [-0.01963403, -0.00528434,  0.02172554, ...,  0.00420537,
        -0.02114365, -0.00322407],
       [-0.02408432,  0.01759911,  0.01289755, ...,  0.01116083,
        -0.02765297,  0.03028362],
       ...,
       [-0.02148633,  0.01786764,  0.01300819, ..., -0.02138482,
        -0.04399392,  0.010988  ],
       [-0.0025764 ,  0.02511008,  0.02134404, ..., -0.02878903,
        -0.04057083,  0.01681587],
       [-0.0326599 ,  0.0177589 ,  0.01950189, ..., -0.00672941,
        -0.05072301, -0.00034183]], dtype=float32)>

In [10]:
tf.abs(ftr)/5

<tf.Tensor: id=65007, shape=(126, 771), dtype=float32, numpy=
array([[0.0606065 , 0.05738253, 0.04570109, ..., 0.2018005 , 0.20329157,
        0.3033479 ],
       [0.05667256, 0.02031567, 0.15214147, ..., 0.22820044, 0.19565551,
        0.12245405],
       [0.02217955, 0.05234206, 0.09567282, ..., 0.23867507, 0.31390697,
        0.26440674],
       ...,
       [1.0093615 , 1.1732726 , 1.0464318 , ..., 0.44613034, 0.44151035,
        0.38526854],
       [1.0093615 , 1.1732726 , 1.0464318 , ..., 0.44613034, 0.44151035,
        0.38526854],
       [1.0093615 , 1.1732726 , 1.0464318 , ..., 0.38518018, 0.38347208,
        0.38763672]], dtype=float32)>

In [11]:
hidden1_b

array([[ 2.2541523,  5.6070485,  7.475915 , ...,  7.5906425,  7.4660616,
         4.714741 ],
       [ 5.805717 , 10.343396 , 13.01164  , ...,  8.346055 ,  8.134406 ,
         5.0012155],
       [ 5.2506895,  9.602298 , 12.362278 , ...,  8.404636 ,  8.317651 ,
         5.233562 ],
       ...,
       [18.567364 , 27.930397 , 30.374174 , ..., 15.384658 , 15.095654 ,
         9.432099 ],
       [18.567364 , 27.930397 , 30.374174 , ..., 15.384658 , 15.095654 ,
         9.432099 ],
       [12.165686 , 19.746794 , 24.37861  , ..., 11.276628 , 11.496893 ,
         7.4698954]], dtype=float32)

In [12]:
a=tf.constant([1,2,3])
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    label_numpy = a.eval()

NotImplementedError: eval is not supported when eager execution is enabled, is .numpy() what you're looking for?

In [ ]:
import h5py 
import tensorflow as tf
hh = h5py.File('ftr_refrmd_10h.hdf5', 'r')
d=hh['ftr_refrmd_10h'][0]
len_data=d.shape
hh.close()
len_data

In [ ]:
file.close()

In [ ]:
x = tf.constant([True, False])
x=tf.cast(x, tf.int32)

In [ ]:
x